## Imports

We are using a typical data science stack: `numpy`, `pandas`, `sklearn`, `matplotlib`. 

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Training data
app_train_domain = pd.read_csv('../input/app_train_domain.csv')
print('Training data shape: ', app_train_domain.shape)
app_train_domain.head()

Training data shape:  (307511, 245)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET,DAYS_EMPLOYED_ANOM,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,1,0,1,0,1,False,2.007889,0.121978,0.060749,-0.067329
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,0,1,0,0,False,4.790750,0.132217,0.027598,-0.070862
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,0,False,2.000000,0.100000,0.050000,-0.011814
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,0,False,2.316167,0.219900,0.094941,-0.159905
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,0,False,4.222222,0.179963,0.042623,-0.152418


The training data has 307511 observations (each one a separate loan) and 122 features (variables) including the `TARGET` (the label we want to predict).

In [3]:
# Testing data features
app_test_domain = pd.read_csv('../input/app_test_domain.csv')
print('Testing data shape: ', app_test_domain.shape)
app_test_domain.head()

Testing data shape:  (48744, 244)


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,100001,0,0,1,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,0,1,0,1,0,False,4.213333,0.152300,0.036147,0.121044
1,100005,0,0,1,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,0,0,0,0,0,False,2.250182,0.175455,0.077973,0.247398
2,100013,0,1,1,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,0,0,0,0,0,False,3.275378,0.344578,0.105202,0.222477
3,100028,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,1,0,0,1,0,False,5.000000,0.155614,0.031123,0.133515
4,100038,0,1,0,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,0,0,0,0,0,False,3.475000,0.178150,0.051266,0.168021


The test set is considerably smaller and lacks a `TARGET` column. 

In [4]:
from sklearn.preprocessing import MinMaxScaler
# imputer for handling missing values
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

#Extract ids
loan_ids = app_train_domain['SK_ID_CURR']
app_train_domain = app_train_domain.drop(columns='SK_ID_CURR')


# Feature names
features = list(app_train_domain.columns)

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(app_train_domain)

# Transform training data
train = imputer.transform(app_train_domain)


# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)



#Split the train data in train and test set
X_train, X_test= train_test_split(train,test_size=0.2,random_state=1)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train.columns = features
X_test.columns = features

#X_train_labels = X_train['TARGET']
#X_train = X_train.drop(columns = ['TARGET'])
#X_test_labels = X_test['TARGET']
#X_test = X_test.drop(columns = ['TARGET'])

#features.remove('TARGET')


print('Training data shape: ', X_train.shape)
print('Testing data shape: ', X_test.shape)

Training data shape:  (246008, 244)
Testing data shape:  (61503, 244)


In [20]:
def own_eval_metric(y_true, y_pred):
    cost = 0
    for i in range(0,len(y_true)):
        if y_pred[i]<=0.5:
            y_pred[i] = 0
        else:
            y_pred[i] = 1
        if(y_true[i]==0 and y_pred[i]==0) or (y_true[i]==1 and y_pred[i]==1):
            cost += 0
        elif (y_true[i]==0 and y_pred[i]==1):
            cost += 1
        else:
            cost +=5
    return ('own_eval_metric',cost, False)

# Just for Fun: Light Gradient Boosting Machine

Now (if you want, this part is entirely optional) we can step off the deep end and use a real machine learning model: the [gradient boosting machine](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/) using the [LightGBM library](http://lightgbm.readthedocs.io/en/latest/Quick-Start.html)! The Gradient Boosting Machine is currently the leading model for learning on structured datasets (especially on Kaggle) and we will probably need some form of this model to do well in the competition. Don't worry, even if this code looks intimidating, it's just a series of small steps that build up to a complete model. I added this code just to show what may be in store for this project, and because it gets us a slightly better score on the leaderboard. In future notebooks we will see how to work with more advanced models (which mostly means adapting existing code to make it work better), feature engineering, and feature selection. See you in the next notebook!  

In [23]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    #train_ids = features['SK_ID_CURR']
    #test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    test_labels = test_features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['TARGET'])
    test_features = test_features.drop(columns = ['TARGET'])
    
  
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    #k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    #out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    #for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        #train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        #valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
    model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                    learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
    model.fit(features, labels, eval_metric = own_eval_metric,
                  eval_set = [ (features, labels)],
                  eval_names = [ 'train'], #categorical_feature = cat_indices,
                  early_stopping_rounds = 10, verbose = 200)
             
        
        # Record the best iteration
        #best_iteration = model.best_iteration_
        
        # Record the feature importances
    feature_importance_values = model.feature_importances_ #/ k_fold.n_splits
        
        # Make predictions
    test_predictions = model.predict_proba(test_features)[:, 1]# / k_fold.n_splits
        
        # Record the out of fold predictions
        #out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        #valid_score = model.best_score_['valid']['auc']
        #train_score = model.best_score_['train']['auc']
        
        #valid_scores.append(valid_score)
        #train_scores.append(train_score)
        
        # Clean up memory
        #gc.enable()
        #del model, train_features, valid_features
        #gc.collect()
        
    # Make the submission dataframe
    #submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    #valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    #valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    #fold_names = list(range(n_folds))
    #fold_names.append('overall')
    
    # Dataframe of validation scores
    #metrics = pd.DataFrame({'fold': fold_names,
     #                       'train': train_scores,
      #                      'valid': valid_scores}) 
    print(test_predictions)
    return  feature_importances, test_predictions

In [24]:
fi, prediction = model(X_train, X_test)
print('Baseline metrics')
print(prediction)

Training Data Shape:  (246008, 243)
Testing Data Shape:  (61503, 243)
[0. 0. 0. ... 0. 0. 0.]
[0.07993465 0.07891661 0.07891661 ... 0.07823385 0.07823385 0.07823385]
[1]	train's binary_logloss: 0.277745	train's own_eval_metric: 99275
[0. 0. 0. ... 0. 0. 0.]
[0.07838592 0.07721409 0.07721409 ... 0.0758804  0.0758804  0.0758804 ]
[2]	train's binary_logloss: 0.275403	train's own_eval_metric: 99275
[0. 0. 0. ... 0. 0. 0.]
[0.07683956 0.07556888 0.07556888 ... 0.07340858 0.07416801 0.07416801]
[3]	train's binary_logloss: 0.273365	train's own_eval_metric: 99275
[0. 0. 0. ... 0. 0. 0.]
[0.0755992  0.0741723  0.07341079 ... 0.07105801 0.07259342 0.07259342]
[4]	train's binary_logloss: 0.271564	train's own_eval_metric: 99275
[0. 0. 0. ... 0. 0. 0.]
[0.07502642 0.07360947 0.07148819 ... 0.06902826 0.07052314 0.07052314]
[5]	train's binary_logloss: 0.26994	train's own_eval_metric: 99275
[0. 0. 0. ... 0. 0. 0.]
[0.07317254 0.07310488 0.06971515 ... 0.0671109  0.06856731 0.06856731]
[6]	train's bin

[50]	train's binary_logloss: 0.247033	train's own_eval_metric: 99237
[0. 0. 0. ... 0. 0. 0.]
[0.0406054  0.05282889 0.0341481  ... 0.02909371 0.05626038 0.03888654]
[51]	train's binary_logloss: 0.246838	train's own_eval_metric: 99233
[0. 0. 0. ... 0. 0. 0.]
[0.04007018 0.05296    0.03375473 ... 0.02879544 0.0563995  0.0390544 ]
[52]	train's binary_logloss: 0.246653	train's own_eval_metric: 99213
[0. 0. 0. ... 0. 0. 0.]
[0.03953881 0.05274636 0.03312186 ... 0.02874918 0.05753964 0.03951095]
[53]	train's binary_logloss: 0.246475	train's own_eval_metric: 99199
[0. 0. 0. ... 0. 0. 0.]
[0.03850187 0.05230236 0.0329849  ... 0.02886878 0.0579061  0.03896244]
[54]	train's binary_logloss: 0.24629	train's own_eval_metric: 99184
[0. 0. 0. ... 0. 0. 0.]
[0.03861835 0.05192882 0.03274456 ... 0.02856754 0.05731976 0.03902259]
[55]	train's binary_logloss: 0.246096	train's own_eval_metric: 99180
[0. 0. 0. ... 0. 0. 0.]
[0.03792487 0.0516232  0.03200269 ... 0.02837775 0.05698431 0.03980392]
[56]	train'

[100]	train's binary_logloss: 0.240106	train's own_eval_metric: 98250
[0. 0. 0. ... 0. 0. 0.]
[0.02762604 0.04349353 0.02444788 ... 0.0253482  0.05585862 0.04064998]
[101]	train's binary_logloss: 0.24002	train's own_eval_metric: 98232
[0. 0. 0. ... 0. 0. 0.]
[0.02754813 0.04362703 0.02450668 ... 0.02533227 0.05567084 0.04077513]
[102]	train's binary_logloss: 0.239931	train's own_eval_metric: 98214
[0. 0. 0. ... 0. 0. 0.]
[0.0271314  0.04357554 0.02424814 ... 0.02543969 0.05585494 0.04124871]
[103]	train's binary_logloss: 0.239839	train's own_eval_metric: 98188
[0. 0. 0. ... 0. 0. 0.]
[0.02752737 0.0434622  0.0241838  ... 0.02525186 0.05598713 0.04114116]
[104]	train's binary_logloss: 0.239758	train's own_eval_metric: 98162
[0. 0. 0. ... 0. 0. 0.]
[0.02758564 0.04333891 0.02424231 ... 0.02494901 0.0561983  0.04064247]
[105]	train's binary_logloss: 0.239672	train's own_eval_metric: 98163
[0. 0. 0. ... 0. 0. 0.]
[0.02718704 0.04333629 0.02426893 ... 0.02497638 0.05552974 0.04068634]
[106]

[150]	train's binary_logloss: 0.235938	train's own_eval_metric: 97583
[0. 0. 0. ... 0. 0. 0.]
[0.02700311 0.04122163 0.02196422 ... 0.02435228 0.05427105 0.04032494]
[151]	train's binary_logloss: 0.235875	train's own_eval_metric: 97579
[0. 0. 0. ... 0. 0. 0.]
[0.02673089 0.04122347 0.02196522 ... 0.02435339 0.05427344 0.04032674]
[152]	train's binary_logloss: 0.235805	train's own_eval_metric: 97564
[0. 0. 0. ... 0. 0. 0.]
[0.0266479  0.04109739 0.02194266 ... 0.02432844 0.05445375 0.0402861 ]
[153]	train's binary_logloss: 0.235726	train's own_eval_metric: 97553
[0. 0. 0. ... 0. 0. 0.]
[0.02665018 0.04110085 0.02194455 ... 0.02442723 0.05445828 0.0396137 ]
[154]	train's binary_logloss: 0.235656	train's own_eval_metric: 97536
[0. 0. 0. ... 0. 0. 0.]
[0.0263758  0.04066768 0.0217087  ... 0.02451828 0.054655   0.03921122]
[155]	train's binary_logloss: 0.235576	train's own_eval_metric: 97515
[0. 0. 0. ... 0. 0. 0.]
[0.02639712 0.04070008 0.02172633 ... 0.02453814 0.05469791 0.03924251]
[156

[200]	train's binary_logloss: 0.232712	train's own_eval_metric: 97148
[0. 0. 0. ... 0. 0. 0.]
[0.02717502 0.04073569 0.02026382 ... 0.02400046 0.0551009  0.04003132]
[201]	train's binary_logloss: 0.232641	train's own_eval_metric: 97148
[0. 0. 0. ... 0. 0. 0.]
[0.02718995 0.04075776 0.02027503 ... 0.02401369 0.05513031 0.04005303]
[202]	train's binary_logloss: 0.232575	train's own_eval_metric: 97143
[0. 0. 0. ... 0. 0. 0.]
[0.02719655 0.04076751 0.02027999 ... 0.02401954 0.05514331 0.04006262]
[203]	train's binary_logloss: 0.232511	train's own_eval_metric: 97143
[0. 0. 0. ... 0. 0. 0.]
[0.02710098 0.04079195 0.0202924  ... 0.02403261 0.05517586 0.04008665]
[204]	train's binary_logloss: 0.232454	train's own_eval_metric: 97124
[0. 0. 0. ... 0. 0. 0.]
[0.02710454 0.04079722 0.02029508 ... 0.02403577 0.05518289 0.04009184]
[205]	train's binary_logloss: 0.232406	train's own_eval_metric: 97124
[0. 0. 0. ... 0. 0. 0.]
[0.02709835 0.04078805 0.02029042 ... 0.02403027 0.05517067 0.04008282]
[206

...

[9803]	train's binary_logloss: 0.0532205	train's own_eval_metric: 12763
[0. 0. 0. ... 0. 0. 0.]
[0.00297921 0.01737717 0.00097042 ... 0.00693975 0.07533135 0.01596702]
[9804]	train's binary_logloss: 0.0532121	train's own_eval_metric: 12758
[0. 0. 0. ... 0. 0. 0.]
[0.00297936 0.01732031 0.00097047 ... 0.0069401  0.07517647 0.01596781]
[9805]	train's binary_logloss: 0.053203	train's own_eval_metric: 12753
[0. 0. 0. ... 0. 0. 0.]
[0.00297946 0.0173209  0.0009705  ... 0.00694033 0.07517888 0.01596836]
[9806]	train's binary_logloss: 0.0531966	train's own_eval_metric: 12753
[0. 0. 0. ... 0. 0. 0.]
[0.00297941 0.01732061 0.00097049 ... 0.00694022 0.07517769 0.01596809]
[9807]	train's binary_logloss: 0.0531905	train's own_eval_metric: 12743
[0. 0. 0. ... 0. 0. 0.]
[0.00297914 0.01732744 0.0009704  ... 0.00693959 0.07190755 0.01596666]
[9808]	train's binary_logloss: 0.0531854	train's own_eval_metric: 12738
[0. 0. 0. ... 0. 0. 0.]
[0.00297937 0.01732873 0.00097047 ... 0.00694011 0.0719152  0.015

[9852]	train's binary_logloss: 0.0528965	train's own_eval_metric: 12563
[0. 0. 0. ... 0. 0. 0.]
[0.00286838 0.01713161 0.00097867 ... 0.00691683 0.07185671 0.01598698]
[9853]	train's binary_logloss: 0.0528946	train's own_eval_metric: 12563
[0. 0. 0. ... 0. 0. 0.]
[0.00286809 0.01712747 0.00097843 ... 0.00691614 0.07185004 0.0159854 ]
[9854]	train's binary_logloss: 0.0528847	train's own_eval_metric: 12558
[0. 0. 0. ... 0. 0. 0.]
[0.00286802 0.01712705 0.0009784  ... 0.00691597 0.07184841 0.01598502]
[9855]	train's binary_logloss: 0.0528796	train's own_eval_metric: 12563
[0. 0. 0. ... 0. 0. 0.]
[0.00286808 0.01712737 0.00097842 ... 0.0069161  0.07184967 0.01598532]
[9856]	train's binary_logloss: 0.0528741	train's own_eval_metric: 12558
[0. 0. 0. ... 0. 0. 0.]
[0.00286804 0.01712718 0.00097841 ... 0.00691602 0.07184891 0.01598513]
[9857]	train's binary_logloss: 0.0528711	train's own_eval_metric: 12563
[0. 0. 0. ... 0. 0. 0.]
[0.00286829 0.01711049 0.00097744 ... 0.00690922 0.07185467 0.01

[9901]	train's binary_logloss: 0.0525941	train's own_eval_metric: 12428
[0. 0. 0. ... 0. 0. 0.]
[0.00288575 0.01705316 0.00095667 ... 0.0069111  0.07195652 0.01599897]
[9902]	train's binary_logloss: 0.0525924	train's own_eval_metric: 12428
[0. 0. 0. ... 0. 0. 0.]
[0.00288569 0.01705279 0.00095665 ... 0.00691095 0.07195505 0.01599862]
[9903]	train's binary_logloss: 0.0525846	train's own_eval_metric: 12423
[0. 0. 0. ... 0. 0. 0.]
[0.00288597 0.01705442 0.00095674 ... 0.00691162 0.07196154 0.01600015]
[9904]	train's binary_logloss: 0.0525771	train's own_eval_metric: 12418
[0. 0. 0. ... 0. 0. 0.]
[0.00288559 0.01705174 0.00095662 ... 0.00691071 0.07195087 0.01599531]
[9905]	train's binary_logloss: 0.0525639	train's own_eval_metric: 12393
[0. 0. 0. ... 0. 0. 0.]
[0.00282351 0.01705058 0.00096158 ... 0.00691023 0.07194624 0.01599422]
[9906]	train's binary_logloss: 0.0525517	train's own_eval_metric: 12378
[0. 0. 0. ... 0. 0. 0.]
[0.00282483 0.01704876 0.00096147 ... 0.00690949 0.07197753 0.01

[9950]	train's binary_logloss: 0.0523156	train's own_eval_metric: 12313
[0. 0. 0. ... 0. 0. 0.]
[0.00269818 0.01704441 0.00097718 ... 0.0069106  0.07140923 0.01598108]
[9951]	train's binary_logloss: 0.0523134	train's own_eval_metric: 12313
[0. 0. 0. ... 0. 0. 0.]
[0.00269813 0.01704414 0.00096888 ... 0.00691048 0.07140814 0.0160491 ]
[9952]	train's binary_logloss: 0.0523044	train's own_eval_metric: 12308
[0. 0. 0. ... 0. 0. 0.]
[0.00269787 0.0170006  0.00097369 ... 0.00690982 0.07140168 0.01604756]
[9953]	train's binary_logloss: 0.0522944	train's own_eval_metric: 12293
[0. 0. 0. ... 0. 0. 0.]
[0.00269768 0.01699943 0.00097362 ... 0.00690934 0.07139705 0.01604646]
[9954]	train's binary_logloss: 0.0522865	train's own_eval_metric: 12293
[0. 0. 0. ... 0. 0. 0.]
[0.00269746 0.01699805 0.00097354 ... 0.0069079  0.07139159 0.01604516]
[9955]	train's binary_logloss: 0.0522764	train's own_eval_metric: 12283
[0. 0. 0. ... 0. 0. 0.]
[0.00269751 0.01699839 0.00097356 ... 0.00690803 0.07139291 0.01

[9999]	train's binary_logloss: 0.0519348	train's own_eval_metric: 12083
[0. 0. 0. ... 0. 0. 0.]
[0.0025946  0.01709663 0.0009734  ... 0.00692751 0.06834704 0.01601862]
[10000]	train's binary_logloss: 0.0519303	train's own_eval_metric: 12083
[0.03015071 0.01709164 0.00562683 ... 0.02714577 0.00167695 0.24934147]
Baseline metrics
[0.03015071 0.01709164 0.00562683 ... 0.02714577 0.00167695 0.24934147]


In [8]:
#submission.to_csv('baseline_lgb.csv', index = False)

Again, we see tha some of our features made it into the most important. Going forward, we will need to think about whatother domain knowledge features may be useful for this problem (or we should consult someone who knows more about the financial industry! 

## Model Interpretation: Feature Importances

As a simple method to see which variables are the most relevant, we can look at the feature importances of the random forest. Given the correlations we saw in the exploratory data analysis, we should expect that the most important features are the `EXT_SOURCE` and the `DAYS_BIRTH`. We may use these feature importances as a method of dimensionality reduction in future work.

In [9]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [10]:
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

NameError: name 'feature_importances' is not defined

In [ ]:
fi_sorted = plot_feature_importances(fi)